### Download the data

In [63]:
!wget https://raw.githubusercontent.com/suvigyajain0101/CaseStudies/main/AdverseEventClassification/Data/AE_Data.csv

--2022-08-18 21:10:19--  https://raw.githubusercontent.com/suvigyajain0101/CaseStudies/main/AdverseEventClassification/Data/AE_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5998096 (5.7M) [text/plain]
Saving to: ‘AE_Data.csv.5’

AE_Data.csv.5       100%[===================>]   5.72M  --.-KB/s    in 0.1s    

2022-08-18 21:10:19 (47.8 MB/s) - ‘AE_Data.csv.5’ saved [5998096/5998096]



### Import Libraries

In [64]:
import pandas as pd
import numpy as np
import re

In [65]:
WORDS_TO_REMOVE = ['##padding##', 'ti-', 'ti -']

In [66]:
df = pd.read_csv('/content/AE_Data.csv')
df.head()

,title,abstract,label
0,antimicrobial impacts of essential oils on foo...,the antimicrobial activity of twelve essential...,0
1,purification and characterization of a cystein...,antimicrobial peptide (amp) crustin is a type ...,0
2,telavancin activity tested against gram-positi...,objectives: to reassess the activity of telava...,0
3,the in vitro antimicrobial activity of cymbopo...,background: it is well known that cymbopogon (...,0
4,screening currency notes for microbial pathoge...,fomites are a well-known source of microbial i...,0


In [67]:
df['label'].value_counts()

0    3851
1     294
Name: label, dtype: int64

### Data Cleaning

1. Combine Title and Abstract
2. Lower case entire corpus
2. Remove newline and tabs from the dataset
3. Remove brackets, #, colons, 'TI" (title identifier), '##PADDING##'
5. Lemmatize and remove stopwords
2. Remove records with less than 10 words

In [68]:
df['text'] = df['title'] + ' ' + df['abstract']
df.head()

,title,abstract,label,text
0,antimicrobial impacts of essential oils on foo...,the antimicrobial activity of twelve essential...,0,antimicrobial impacts of essential oils on foo...
1,purification and characterization of a cystein...,antimicrobial peptide (amp) crustin is a type ...,0,purification and characterization of a cystein...
2,telavancin activity tested against gram-positi...,objectives: to reassess the activity of telava...,0,telavancin activity tested against gram-positi...
3,the in vitro antimicrobial activity of cymbopo...,background: it is well known that cymbopogon (...,0,the in vitro antimicrobial activity of cymbopo...
4,screening currency notes for microbial pathoge...,fomites are a well-known source of microbial i...,0,screening currency notes for microbial pathoge...


In [69]:
df.replace(r'\n','', regex=True).iloc[4140, :]['text']

'TI  - [AUTO-INFECTION (INTESTINAL) IN RADIATION SICKNESS AND ITS PREVENTION IN WISTAR WHITE RATS]. ##PADDING##'

In [70]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [71]:
eng_stopwords = stopwords.words('english')
stemmer = WordNetLemmatizer()

joined_words_to_remove = '|'.join(WORDS_TO_REMOVE)


def clean_text(x):
  # Lower case the text
  lower_x = x.lower()

  # Remove line breaks and tabs
  no_break_x = re.sub("\n|\r|\t", " ", lower_x)

  # Remove specific words
  no_waste_words_x = re.sub(joined_words_to_remove, " ", no_break_x)

  # Remove all non alphabet, numeral and space characters
  alpha_x = re.sub('[^0-9a-zA-Z ]+', ' ', no_waste_words_x)

  # Remove stopwords and lemmatize the word. Join at the end will also remove multi-spaces
  lemma_x = ' '.join([stemmer.lemmatize(word) for word in alpha_x.split() if word not in eng_stopwords])

  return lemma_x

Let's test the function on few examples

In [72]:
for sample_text in df.sample(5)['text'].values:
  print('ORIGINAL TEXT : ', sample_text)
  print('-'*100)
  print('CLEANED TEXT : ', clean_text(sample_text))
  print('\n')
  print('*'*100)

ORIGINAL TEXT :  transferable plasmid-mediated antibiotic resistance inother_species.
 a strain ofother_species, isolated from a patient with meningoencephalitis, was resistant to chloramphenicol, erythromycin, streptomycin, and tetracycline. the genes conferring resistance to these antibiotics were carried by a 37-kb plasmid, pip811, that was self-transferable to other l monocytogenes cells, to enterococci-streptococci, and toother_species. the efficacy of transfer and the stability of pip811 were  higher in enterococci-streptococci than in the other gram-positive bacteria. as indicated by nucleic acid hybridisation, the genes in pip811 conferring resistance to chloramphenicol, erythromycin, and streptomycin were closely related to plasmid-borne determinants that are common in enterococci-streptococci. plasmid pip811 shared extensive sequence homology with  pam beta 1, the prototype broad host range resistance plasmid in these two groups of gram-positive cocci. these results suggest t

In [73]:
# Apply cleaning function to the text field
df['clean_text'] = df['text'].apply(lambda x : clean_text(x))

# Get the length and drop records less than 10 words
df['text_len'] = df['clean_text'].str.split().apply(len)

cleaned_df = df[df['text_len'] > 10][['clean_text', 'label']]

In [74]:
cleaned_df.head()

,clean_text,label
0,antimicrobial impact essential oil food borne ...,0
1,purification characterization cysteine rich 14...,0
2,telavancin activity tested gram positive clini...,0
3,vitro antimicrobial activity cymbopogon essent...,0
4,screening currency note microbial pathogen ant...,0


In [75]:
print('Total records retained after data cleaning : ', cleaned_df.shape[0])
cleaned_df['label'].value_counts()

Total records retained after data cleaning :  4013


0    3719
1     294
Name: label, dtype: int64

### TF-IDF Vectorizer

Convert text to features. We'll use TF-IDF score to give the score to the word in the corpus. 

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(cleaned_df['clean_text']).toarray()
y = cleaned_df['label'].values

Since the data is unbalanced, we need to split the data into train-test in such a way that those represent the actual data. That's where stratified sampling comes in

In [99]:
from sklearn.model_selection import train_test_split

test_split = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, stratify=y)

In [106]:
print('Label Distribution in the training data')
print(np.unique(y_train, return_counts=True))
print('*'*50)
print('Label Distribution in the testing data')
print(np.unique(y_test, return_counts=True))

Label Distribution in the training data
(array([0, 1]), array([2975,  235]))
**************************************************
Label Distribution in the testing data
(array([0, 1]), array([744,  59]))


Now that the text has been converted into features, we can model the data 

## Machine Learning Models

The dataset at hand is so imbalanced that accuracy on the predictions is not a good metric to judge a model. We'll use Classification Report, and more importantly F1 Score for model comparison. Also to note, we want to reduce False negatives as much as we can, we don't want to classify a doc non-adverse event if it's in fact a adverse-event related

In [107]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Experiment 1 - Random Forest

In [109]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)

# Fit the model to the data
classifier.fit(X_train, y_train)

# Generate predictions
y_pred = classifier.predict(X_test)

# Prepare Classification Report
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[743   1]
 [ 35  24]]


              precision    recall  f1-score   support

           0       0.96      1.00      0.98       744
           1       0.96      0.41      0.57        59

    accuracy                           0.96       803
   macro avg       0.96      0.70      0.77       803
weighted avg       0.96      0.96      0.95       803



Performance on 0s is satisfactory, but 1s are pretty terrible!

### Experiment 2 - Multinomial NB

In [110]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the model
classifier = MultinomialNB()

# Fit the model to the data
classifier.fit(X_train, y_train)

# Generate predictions
y_pred = classifier.predict(X_test)

# Prepare Classification Report
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[742   2]
 [ 39  20]]


              precision    recall  f1-score   support

           0       0.95      1.00      0.97       744
           1       0.91      0.34      0.49        59

    accuracy                           0.95       803
   macro avg       0.93      0.67      0.73       803
weighted avg       0.95      0.95      0.94       803



Even worse!